In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from ultralytics.nn.modules.block import C2f
from ultralytics import YOLO


class ChannelAttention(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.fc = nn.Sequential(
            nn.Conv2d(in_channels, in_channels // reduction, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_channels // reduction, in_channels, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()


    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x_cat = torch.cat([avg_out, max_out], dim=1)
        return self.sigmoid(self.conv(x_cat))

class CBAM(nn.Module):
    def __init__(self, channels, reduction=16):
        super(CBAM, self).__init__()
        self.ca = ChannelAttention(channels, reduction)
        self.sa = SpatialAttention()

    def forward(self, x):
        x = x * self.ca(x)
        x = x * self.sa(x)
        return x


In [2]:
class C2fCBAM(nn.Module):
    def __init__(self, original_c2f):
        super(C2fCBAM, self).__init__()
        self.c2f = original_c2f
        out_channels = original_c2f.cv2.conv.out_channels  
        self.cbam = CBAM(out_channels)

    def forward(self, x):
        x = self.c2f(x)
        return self.cbam(x)

In [3]:
def inject_cbam_blocks(model):
    for i, layer in enumerate(model.model):
        if isinstance(layer, C2f) and i >= 5:  
            model.model[i] = C2fCBAM(layer)
    return model

In [4]:
def train_model():
    model = YOLO("yolov8n.pt")
    model.model = inject_cbam_blocks(model.model)

    model.train(
        data="/home/zaman/Code/Breast_Cancer_Detection_and_Monitoring/Cancer.v1i.yolov8/data.yaml",
        epochs=100,
        imgsz=640,
        batch=4,
        optimizer="AdamW",
        lr0=0.0005,
        weight_decay=0.0005,
        amp=True,
        device=0,
        save=True
    )

    return model

trained_model = train_model()


New https://pypi.org/project/ultralytics/8.3.185 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.165 🚀 Python-3.10.12 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/zaman/Code/Breast_Cancer_Detection_and_Monitoring/Cancer.v1i.yolov8/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1

train: Scanning /home/zaman/Code/Breast_Cancer_Detection_and_Monitoring/Cancer.v1i.yolov8/train/labels... 652 images, 0 backgrounds, 0 corrupt: 100%|██████████| 652/652 [00:00<00:00, 1040.52it/s]

train: New cache created: /home/zaman/Code/Breast_Cancer_Detection_and_Monitoring/Cancer.v1i.yolov8/train/labels.cache


val: Fast image access ✅ (ping: 0.1±0.2 ms, read: 159.6±39.6 MB/s, size: 55.6 KB)


val: Scanning /home/zaman/Code/Breast_Cancer_Detection_and_Monitoring/Cancer.v1i.yolov8/valid/labels... 135 images, 0 backgrounds, 0 corrupt: 100%|██████████| 135/135 [00:00<00:00, 553.00it/s]

...

      1/100     0.559G      2.367      3.113      2.717          9        640: 100%|██████████| 163/163 [00:18<00:00,  8.94it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:01<00:00, 10.49it/s]

                   all        135        135      0.587      0.264     0.0757     0.0213



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      2/100     0.672G      1.371      2.207      1.859         10        640:  96%|█████████▋| 157/163 [00:17<00:00,  9.14it/s]
Exception in thread Thread-17 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/zaman/.local/lib/python3.10/site-packages/torch/utils/data/_utils/pin_memory.py", line 61, in _pin_memory_loop
    do_one_step()
  File "/home/zaman/.local/lib/python3.10/site-packages/torch/utils/data/_utils/pin_memory.py", line 37, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
  File "/home/zaman/.local/lib/python3.10/site-packages/torch/multiprocessing/reductions.py", line 541, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/li

KeyboardInterrupt: 